Today we will do something a bit closer to robots and talk about commands and the command loop in the
WPI library (wpilib) that is the framework for all of our robot coding. We will not be using the
wpilib code, but we will write a little bit of code to demonstrate
concepts. The first thing we need is a command and a command loop.

In wpilib commands run every 20 milliseconds, so we need a `Command` class so we can make `Command` objects,
and then a loop that will run a list of `Command` objects.

# A Very Simple `Command` Class

below is a very simple start on a `Command` class. The only member variable is a name, and during construction we set this to be the class name. Additionally, there will be an `execute()` method. For now the execte method will just
print that the command is running.

In [1]:
// This is a Command class, initially we will just start with a constructor,
// naming, and an execute method.
class Command
{
    String m_name;
    Command()
    {
        // By default the name is the class name. If this is called
        // from the constructor of an extending class, then the
        // class name will be the name of the extending class.
        m_name = getClass().getName();
        m_name = m_name.substring(m_name.lastIndexOf('$') + 1);
    }
    void execute()
    {
        System.out.println("executing " + m_name);
    }
}

// create and run a command to make sure it works
Command command = new Command();
command.execute();


executing Command


# Building a Command Loop

And now we need a command loop that will hold a list of commands and run those
commands every 20ms (what wpilib does by default). Let's work up to this in steps. The first
thing we will do is build a simple loop that runs for some amount of time (like the
15sec autonomous period). We will start by getting the current time, adding the amount of
time we want the loop to run to get the end time, and then testing that the current
time is less than the end time for every loop:

In [13]:
long RUN_TIME = 30;

long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
while (System.currentTimeMillis() < endTime)
{
    System.out.println("time = " + System.currentTimeMillis());

}

startTime = 1626390482713
time = 1626390482737
time = 1626390482737
time = 1626390482737
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482738
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482739
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482740
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1626390482741
time = 1

This is interesting because we notice the time between the start time and the first reported time
in the loop is about 25ms. What this says to me is that the overhead of running Java in the jupyter
notebook is really high. The wpilib uses a 20ms loop time. I'm going to suggest we use a bigger
time for our test code, maybe something like 40ms, so that there is enough time to complete things
between steps in this environment.

In this next step we will add some timing logic so we don't do 10 steps in the same millisecond, but
rather space the steps out to 40ms. Let's add a constant, `STEP_TIME`, set to 40ms, but that we can
tune if necessary (the wpilib does let you set the command execution increment, which you might want
to do if there are so many commands you cannot meet the expected 20ms time for most/all of the robot
control period (autonomous or manual).

For command loop timing we will add a `STEP_TIME` constant, a `nextStepEnd` variable that will increase
by `STEP_TIME` each timen the loop runs, and a bit of code that *sleeps* until the next execution is due
for each step. We will print the expected step time and the actul step time, and see that these almost match:

In [2]:
long RUN_TIME = 400;
long STEP_TIME = 40;

long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
long nextStepEnd = startTime;
while (System.currentTimeMillis() < endTime)
{
    // control of the step time
    long sleepTime = nextStepEnd - System.currentTimeMillis();
    if (sleepTime > 0)
    {
        Thread.sleep(sleepTime);
    }
    System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
    nextStepEnd += STEP_TIME;
}

startTime = 1626799774001
time = 1626799774073; wanted: 1626799774001
time = 1626799774075; wanted: 1626799774041
time = 1626799774081; wanted: 1626799774081
time = 1626799774121; wanted: 1626799774121
time = 1626799774162; wanted: 1626799774161
time = 1626799774202; wanted: 1626799774201
time = 1626799774242; wanted: 1626799774241
time = 1626799774281; wanted: 1626799774281
time = 1626799774321; wanted: 1626799774321
time = 1626799774362; wanted: 1626799774361
time = 1626799774402; wanted: 1626799774401


Why doesn't the actual time match the wanted time exactly match the wanted time? ***Good Question*** - The
reason is that processing every line of code takes a little bit of time. In the interpreted environment
of notebooks this is more time than it would take on the robot because there are lots of layers of code
between the code you type, and having the notebook run that code and return the result - and one of those
is the *build*, which compiles code to a form the robot can easily run - in this environment the build
happens while we run the code - which takes a while on the first time through.

The next step in building the command loop is actually having a set (list) of commands
that should be run in every step, and running them. Java has a number of ways to represent
a collection of something like commands. In this case we can google for a java collection
that implements the Java `List` interface (we will talk about interfaces a bit later).

Yeah, this is a little confusing, but - for our needs, a `List` is something that:
* keeps an ordered list (though ordering is not really important for us);
* lets us iterate and do something (i.e. like `execute()`) to each of the commands on the list;
* lets us add things (i,e, `Command`s) to the list;
* lets us remove things (i.e.`Command`s) from the list.

So, Java has a collection, called
[`ArrayList`](https://docs.oracle.com/javase/8/docs/api/java/util/ArrayList.html), that does all of
this, and this is how we can use it in our command loop:

In [3]:
long RUN_TIME = 400;
long STEP_TIME = 40;

// Setup the timing for running the commands in the loop.
long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
long nextStepEnd = startTime;

// setup the command list, and the commands to run the robot
ArrayList commandList = new ArrayList();
commandList.add(new Command());

while (System.currentTimeMillis() < endTime)
{
    // control of the step time
    long sleepTime = nextStepEnd - System.currentTimeMillis();
    if (sleepTime > 0)
    {
        Thread.sleep(sleepTime);
    }
    System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
    nextStepEnd += STEP_TIME;
    
    Iterator<Command> iter = commandList.iterator();
    while (iter.hasNext()) 
    {
        Command command = iter.next();
        command.execute();
    }
    
}

startTime = 1626799775879
time = 1626799775978; wanted: 1626799775879
executing Command
time = 1626799775979; wanted: 1626799775919
executing Command
time = 1626799775980; wanted: 1626799775959
executing Command
time = 1626799776000; wanted: 1626799775999
executing Command
time = 1626799776040; wanted: 1626799776039
executing Command
time = 1626799776079; wanted: 1626799776079
executing Command
time = 1626799776120; wanted: 1626799776119
executing Command
time = 1626799776164; wanted: 1626799776159
executing Command
time = 1626799776199; wanted: 1626799776199
executing Command
time = 1626799776240; wanted: 1626799776239
executing Command
time = 1626799776279; wanted: 1626799776279
executing Command


Well that seems pretty good. You might notice that the first time the command runs it is quite a bit later than aexpected, but then is is pretty close after that. In the notebook the Java is being compiled and checked the
first time it is run - which takes a while. Remember that we normally *build* the code, then download it to
the robot. In the notebook environment, all the work of the build happens every time we run the cell - so some things take longer than they would when running on the robot.

# Making the Code Better

Right now we have a `Command` class and a command loop. We really want to use the `Command` class as a
template for building lots of different commands for driving, running the shooter, sensing color, targetting,
etc. We also want to write our robot code in a way that helps other programmers on the team use it correctly
and not break things through misunderstanding intent.

So let's start by making the naming of the `Command` more useful. We will add some methods in the `Command`
class to get and set the name of the command:

In [4]:
// This is a Command class, initially we will just start with a constructor,
// naming, and an execute method.
class Command
{
    private String m_name;
    Command()
    {
        // By default the name is the class name. If this is called
        // from the constructor of an extending class, then the
        // class name will be the name of the extending class.
        m_name = getClass().getName();
        m_name = m_name.substring(m_name.lastIndexOf('$') + 1);
    }
    
    Command setName(String name)
    {
        m_name = name;
        return this;
    }
    
    String getName()
    {
        return m_name;
    }
    
    void execute()
    {
        System.out.println("executing: " + m_name);
    }
}

// create and run a command to make sure it works
Command command = new Command().setName("my new command");
command.execute();


executing: my new command


There are different conventions for writing `get` and `set` methods for classes. Often the `set` command
is typed as a `void`, however, if it returns itself (the `return this;`), then you can *chain* method calls. So,
what exactly does that mean? Well, let's look at this line:
```
Command command = new Command().setName("my new command");
```
If the `setName(String name)' was implemented as:
```
    void setName(String name)
    {
        m_name = name;
    }
```
then the line we are looking at would need to be:
```
Command command = new Command()
command.setName("my new command");
```
Because `setName()` returns the `Command` object we can chain it with `new Command()` which also
returns the `Command` object.

So now that we can set the name, let's run more than 1 command in the loop:

In [5]:
long RUN_TIME = 400;
long STEP_TIME = 40;

// Setup the timing for running the commands in the loop.
long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
long nextStepEnd = startTime;

// setup the command list, and the commands to run the robot
ArrayList commandList = new ArrayList();
commandList.add(new Command().setName("cmd1"));
commandList.add(new Command().setName("cmd2"));

while (System.currentTimeMillis() < endTime)
{
    // control of the step time
    long sleepTime = nextStepEnd - System.currentTimeMillis();
    if (sleepTime > 0)
    {
        Thread.sleep(sleepTime);
    }
    System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
    nextStepEnd += STEP_TIME;
    
    Iterator<Command> iter = commandList.iterator();
    while (iter.hasNext()) 
    {
        Command command = iter.next();
        command.execute();
    }
    
}

startTime = 1626799779400
time = 1626799779503; wanted: 1626799779400
executing: cmd1
executing: cmd2
time = 1626799779504; wanted: 1626799779440
executing: cmd1
executing: cmd2
time = 1626799779504; wanted: 1626799779480
executing: cmd1
executing: cmd2
time = 1626799779520; wanted: 1626799779520
executing: cmd1
executing: cmd2
time = 1626799779565; wanted: 1626799779560
executing: cmd1
executing: cmd2
time = 1626799779601; wanted: 1626799779600
executing: cmd1
executing: cmd2
time = 1626799779641; wanted: 1626799779640
executing: cmd1
executing: cmd2
time = 1626799779680; wanted: 1626799779680
executing: cmd1
executing: cmd2
time = 1626799779722; wanted: 1626799779720
executing: cmd1
executing: cmd2
time = 1626799779763; wanted: 1626799779760
executing: cmd1
executing: cmd2
time = 1626799779802; wanted: 1626799779800
executing: cmd1
executing: cmd2


OK, now we have a command loop, a list of commands to run, and things seem to work fine. But, the code is
a bit *messy* in terms of *REAL* Java conventions. I'll clean this up a bit
and then let's talk about the changes:

In [17]:
long RUN_TIME = 400;
long STEP_TIME = 40;

// Setup the timing for running the commands in the loop.
long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
long nextStepEnd = startTime;

// setup the command list, and the commands to run the robot
List<Command> commandList = new ArrayList<>();
commandList.add(new Command().setName("cmd1"));
commandList.add(new Command().setName("cmd2"));

while (System.currentTimeMillis() < endTime)
{
    // control of the step time
    long sleepTime = nextStepEnd - System.currentTimeMillis();
    if (sleepTime > 0)
    {
        Thread.sleep(sleepTime);
    }
    System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
    nextStepEnd += STEP_TIME;
    
    Iterator<Command> iter = commandList.iterator();
    while (iter.hasNext()) 
    {
        iter.next().execute();
    }
    
}

startTime = 1626802715179
time = 1626802715256; wanted: 1626802715179
executing: cmd1
executing: cmd2
time = 1626802715256; wanted: 1626802715219
executing: cmd1
executing: cmd2
time = 1626802715259; wanted: 1626802715259
executing: cmd1
executing: cmd2
time = 1626802715301; wanted: 1626802715299
executing: cmd1
executing: cmd2
time = 1626802715343; wanted: 1626802715339
executing: cmd1
executing: cmd2
time = 1626802715383; wanted: 1626802715379
executing: cmd1
executing: cmd2
time = 1626802715424; wanted: 1626802715419
executing: cmd1
executing: cmd2
time = 1626802715463; wanted: 1626802715459
executing: cmd1
executing: cmd2
time = 1626802715503; wanted: 1626802715499
executing: cmd1
executing: cmd2
time = 1626802715544; wanted: 1626802715539
executing: cmd1
executing: cmd2
time = 1626802715584; wanted: 1626802715579
executing: cmd1
executing: cmd2


There are some new Java
constructs introduced here, so let's talk about them a bit. The first is the line where we create the
command list - what's happening here?
```
List<Command> commandList = new ArrayList<>();
```
To explain this, let's start with a more simple version, which was:
```
ArrayList commandList = new ArrayList();
```
Which is perfectly acceptable and will work just fine (substitute this and rerun the cell above to check
that). So why do we do something that is seemingly more complicated? The reason is that there a lot of different
ways to write the code for a list, each with different pros and cons usually in terms of memory use,
speed of iteration, and speed of changing what's in the list.

`ArrayList` is the most general purpose implementation, other implementations are
are [`Vector`](https://docs.oracle.com/javase/8/docs/api/java/util/Vector.html)
and [`LinkedList`](https://docs.oracle.com/javase/8/docs/api/java/util/LinkedList.html).

[List](https://docs.oracle.com/javase/8/docs/api/java/util/List.html) is an interface, which we will
talk about in detail a bit later, which is best described as a template for the methods that need to
be written for each implementation of a list. That makes the implementations interchangeable - i.e. you
can replace `ArrayList` with `LinkedList` or `Vector` and the code will work fine.

The `<Command>` in `List<Command>` says that the list will contain `Command` objects. If someone
tries to put something else on the list you will get a build time error before you put the code on
the robot. If you don't include the `<Command>`, then you could write code that put things other than
`Command` and it would cause a runtime error on the robot, which is more difficult to track down and fix.

And last, in the loop, we see the line
```
        iter.next().execute();
```
Which is similar to the chaining example we talked about for the `Command.setName()`. The iterator is
returning a `Command`, so we can call the `execute()` method without creating an intermediate variable.


# Creating a Command

What we have done so far is build a `Command` class that corresponds to the wpilib `CommandBase` class; and some code to execute `Command` objects in a controlled loop, which corresponds to some of the functionality of the wpilib
`CommandScheduler` class. The next thing we are going to do is create a command that does something, in this case,
run for some number of command cycles.

In real robot programming you would never create a `Command` object because it does not do anything useful and it is part of the framework, so you cannot change it. Instead, what we do is extend the functionality of the `Command` by
creating a new `class` for our command that `extends Command`. So let's create a `CountedCommand` that runs for some number of command steps. The first thing to do is create `CountedCommand` class and try to use it as a `Command`:

In [7]:
class CountedCommand extends Command
{
}

// create and run a command to make sure it works
Command command = new CountedCommand();
command.execute();
command.execute();
command.execute();

Command testCommand = new CountedCommand().setName("test CountedCommand");
testCommand.execute();
testCommand.execute();
testCommand.execute();

executing: CountedCommand
executing: CountedCommand
executing: CountedCommand
executing: test CountedCommand
executing: test CountedCommand
executing: test CountedCommand


What is really happening when we extend `Command`? Well, you can think of `Command` as kind of a template
for what needs to be in a command. Since `CountedCommand` extends `Command` it is both a `CountedCommand`
and a `Command`. So this is why we can write the line:
```
Command command = new CountedCommand();
```
and there is no type error. Test what happens if you comment out the `extends Command` as:
```
class CountedCommand // extends Command
```
and you get in `incompatible types` error. Further, if you change the `Command` type
to `CountedCommand` as:
```
CountedCommand command = new CountedCommand();
```
you will get an error that there is no `execute()` method.


This is a demonstration of inheritance, where `Command` is refered to as the *superclass* and `CountedCommand` is
refered to as the *subclass*. The subclass inherits the variables and methods from the superclass unless they are
`private` in the superclass.

In [8]:
class CountedCommand extends Command 
{
    // the step count
    private long m_stepCt = 0;
    // the number of command steps we run before we stop
    final long m_numberOfSteps;
    
    CountedCommand(long numberOfSteps)
    {
        super();
        m_numberOfSteps = numberOfSteps;
    }
    
    @Override
    void execute()
    {
        super.execute();
        m_stepCt += 1;
        System.out.println("  stepCt: " + m_stepCt + " of " + m_numberOfSteps);
    }
}

// create and run a command to make sure it works
Command command = new CountedCommand(10);
command.execute();
command.execute();
command.execute();

Command testCommand = new CountedCommand(10).setName("test CountedCommand");
testCommand.execute();
testCommand.execute();
testCommand.execute();

executing: CountedCommand
  stepCt: 1 of 10
executing: CountedCommand
  stepCt: 2 of 10
executing: CountedCommand
  stepCt: 3 of 10
executing: test CountedCommand
  stepCt: 1 of 10
executing: test CountedCommand
  stepCt: 2 of 10
executing: test CountedCommand
  stepCt: 3 of 10


Note that we have added a variables for the number of steps for the command to run, which gets set
in the constructor; a counter for steps; and an `execute()` method that overrides the `execute()`
method in `Command` to update the step counter each time the command is run. The `@Override` reminds
other programmers this method overides a method in the superclass. Also note that we call
the `super.execute()` so we are making use of the execute functionality from `Command`, and then
extending it to update the counter and print where we are in the step count.

And now, let's add this in running the command loop:

In [18]:
long RUN_TIME = 400;
long STEP_TIME = 40;

// Setup the timing for running the commands in the loop.
long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
long nextStepEnd = startTime;

// setup the command list, and the commands to run the robot
List<Command> commandList = new ArrayList<>();
commandList.add(new Command().setName("cmd1"));
commandList.add(new Command().setName("cmd2"));
commandList.add(new CountedCommand(4));

while (System.currentTimeMillis() < endTime)
{
    // control of the step time
    long sleepTime = nextStepEnd - System.currentTimeMillis();
    if (sleepTime > 0)
    {
        Thread.sleep(sleepTime);
    }
    System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
    nextStepEnd += STEP_TIME;
    
    Iterator<Command> iter = commandList.iterator();
    while (iter.hasNext()) 
    {
        iter.next().execute();
    }
    
}

startTime = 1626802877018
time = 1626802877107; wanted: 1626802877018
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 1 of 4
time = 1626802877108; wanted: 1626802877058
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 2 of 4
time = 1626802877108; wanted: 1626802877098
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 3 of 4
time = 1626802877143; wanted: 1626802877138
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 4 of 4
time = 1626802877183; wanted: 1626802877178
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 5 of 4
time = 1626802877223; wanted: 1626802877218
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 6 of 4
time = 1626802877263; wanted: 1626802877258
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 7 of 4
time = 1626802877303; wanted: 1626802877298
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 8 of 4
time = 1626802877343; 

Well, this is interesting. The `CountedCommand` is running, but it doesn't stop after the expected number
of steps. Guess we need a bit more code. Join us in the next section to work out those details.